In [ ]:
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config

np_config.enable_numpy_behavior()

raw_image_dataset = tf.data.TFRecordDataset('../images.tfrecords')

# Create a dictionary describing the features.
image_feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  example = tf.io.parse_single_example(example_proto, image_feature_description)
  label, image = example["label"], example["image_raw"]
  label = tf.one_hot(label, depth=10)
  # print(tf.io.parse_single_example(example_proto, image_feature_description))
  # image_raw = image_features['image_raw']
  image = tf.io.decode_raw(image, tf.uint8)
  image = tf.reshape(image, (640, 240, 1))
  image = image.astype("float32") / 255
  
  return image, label
  
  # return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

In [ ]:
for image_features in parsed_image_dataset:
  image = image_features[0]
  label = image_features[1]
  print(label)
  print(image)
  break

In [ ]:
dataset = parsed_image_dataset.shuffle(1000).batch(8)

In [ ]:
for batch in dataset:
  print(batch)
  break

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
layer1 = layers.Conv2D(filters=32, input_shape=(640, 240, 1),
  kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')
model.add(layer1)

layer2 = layers.Dense(128, activation='relu')
model.add(layer2)

model.add(layers.Flatten())

layer3 = layers.Dense(10, activation='softmax')
model.add(layer3)

model.summary()

In [ ]:

myadam = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'
)

model.compile(optimizer=myadam,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

# 降低学习率
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',verbose=1, factor=0.2,patience=1, min_lr=0.0001)

# 提前终止
# es=tf.keras.callbacks.EarlyStopping(monitor='loss',verbose=1,mode='min',patience=2)
 
# model.fit(dataset, epochs=10, callbacks=[reduce_lr ,es])
model.fit(dataset, epochs=10, callbacks=[reduce_lr])